In [1]:
import pandas as pd
import json
from collections import defaultdict, Counter
import glob
from analysis_func.text_preproc import preproc_text
import numpy as np
from collections import defaultdict, Counter
import glob
import json
from random import sample
import sklearn

import re
import string
import warnings
from bs4 import BeautifulSoup
import pickle as pkl

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('stsb-bert-large')

from pandarallel import pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
data = pd.read_csv("../lite_data/first_complete_annotated_data.csv", header=0)
data.head()

,Unnamed: 0,author,subreddit,id,title,label
0,0,Background-Dingo6,QAnonCasualties,holleyh,Thanks for sharing. I am gaining some hope I c...,qadjacent
1,1,d-_-bored-_-b,QAnonCasualties,hoiyyc8,Sorry I meant before I left Qanon IIRC it was ...,recovery
2,2,RhinestoneToxidrome,QAnonCasualties,hofdyp2,I’m so sorry that you’re going through this. W...,support
3,3,bigbear3321,QAnonCasualties,ho8pnfl,I remember when I went down the 9/11 rabbit ho...,recovery
4,4,butteredbuttbiscuit,QAnonCasualties,hnr0btg,Lots of people go to places like Tractor Suppl...,irrelevant


In [3]:
print("cleaning data")
#text preproessing - filter engligh, hindi, marathi stop words, remove puncts, hash, mentions, urls, weird spaces etc.
data['clean_text'] = data['title'].parallel_apply(lambda x: preproc_text(x))

cleaning data


In [4]:
data.sample(5)

,Unnamed: 0,author,subreddit,id,title,label,clean_text
470,197,DesignerAmbassador61,ReQovery,lqqdob,Was there a large scale of election fraud that...,irrelevant,large scale election fraud led bidens presiden...
16,16,huntingforkink,QAnonCasualties,hjka5pc,Omg! I'm struggling even THINKING about what t...,irrelevant,omg struggling thinking cook
29,29,Pitiful_Control,QAnonCasualties,hi1cf24,I hear you - I went down with it a couple week...,irrelevant,hear couple weeks ago double vaxxed tightly pa...
484,212,LoveB4action,ReQovery,lx126b,Red-pill for red pilled people ;) Where conspi...,irrelevant,redpill red pilled ;) conspiracy theories love...
432,155,Kialay,ReQovery,l2tgy0,We’ve made this group on telegram for those wh...,irrelevant,we’ve group telegram active supportive communi...


In [5]:
data['bin_label'] = data['label'].apply(lambda x: 1 if x=='recovery' else 0)


In [6]:
corpus = data['clean_text'].tolist()
corpus_embeddings = model.encode(corpus)

sent2embed = defaultdict()

for sentence, embedding in zip(corpus, corpus_embeddings):
    sent2embed[sentence] = embedding
    


with open("../lite_data/initial_bert_embeddings.pkl", "wb") as pfile:
    pkl.dump(sent2embed, pfile)


In [7]:
sentence

'deal dealt success mom deeply involved thinks john kennedy jr alive vaccine poison trump charge years talking substantial amount money worse health deteriorated point walker wheelchair adamantly refuses doctor quality life horrible doesnt listen reason sound frequencies internet thinks sterilizes water light online plays monitor buys healing crystals barely speak upsets dealt medical'

In [8]:
corpus_embeddings

array([[ 0.54952   , -0.28799117, -0.21290419, ..., -0.6227958 ,
         0.7103508 , -0.17310521],
       [ 0.26597312,  0.45821318,  0.24108514, ..., -0.60857385,
         0.18491228,  0.64838904],
       [ 0.7347871 , -0.8543208 ,  0.50194865, ..., -0.2864929 ,
         0.86271197,  0.32289717],
       ...,
       [ 0.28939855, -0.42697808,  0.38912985, ..., -0.22788598,
         0.32963043, -0.17209585],
       [-0.04185219, -0.19233319, -0.28876302, ...,  0.08219387,
         0.579799  , -0.73323464],
       [ 0.4613443 , -1.207446  ,  0.2026291 , ..., -0.1257518 ,
         0.74228096, -0.4386694 ]], dtype=float32)

In [9]:
corpus_embeddings.shape

(614, 1024)

In [10]:
len(data)

614

In [11]:
labels = np.array(data['bin_label'].tolist())

## naive bayes

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, balanced_accuracy_score
import matplotlib.pyplot as plt

In [13]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(corpus_embeddings, labels, test_size=0.3, random_state=i)
    gnb = classifiers[i]
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    print(names[i], ": ", 'Precision: %.3f' % precision_score(y_test, y_pred)),
    print('Recall: %.3f' % recall_score(y_test, y_pred)),
    print('Accuracy: %.3f' % accuracy_score(y_test, y_pred)),
    print('F1 Score: %.3f' % f1_score(y_test, y_pred)),
    print('BAC: %.3f' % balanced_accuracy_score(y_test, y_pred))

Nearest Neighbors :  Precision: 0.552
Recall: 0.667
Accuracy: 0.886
F1 Score: 0.604
BAC: 0.793
Linear SVM :  Precision: 0.708
Recall: 0.773
Accuracy: 0.935
F1 Score: 0.739
BAC: 0.865
RBF SVM :  Precision: 1.000
Recall: 0.625
Accuracy: 0.935
F1 Score: 0.769
BAC: 0.812
Gaussian Process :  Precision: 0.722
Recall: 0.765
Accuracy: 0.903
F1 Score: 0.743
BAC: 0.849
Decision Tree :  Precision: 0.714
Recall: 0.667
Accuracy: 0.903
F1 Score: 0.690
BAC: 0.808
Random Forest :  Precision: 1.000
Recall: 0.171
Accuracy: 0.816
F1 Score: 0.292
BAC: 0.585
Neural Net :  Precision: 0.846
Recall: 0.971
Accuracy: 0.962
F1 Score: 0.904
BAC: 0.965
AdaBoost :  Precision: 0.844
Recall: 0.750
Accuracy: 0.924
F1 Score: 0.794
BAC: 0.858
Naive Bayes :  Precision: 0.441
Recall: 0.812
Accuracy: 0.789
F1 Score: 0.571
BAC: 0.798
QDA :  Precision: 0.526
Recall: 0.909
Accuracy: 0.838
F1 Score: 0.667
BAC: 0.866


/home/phadke/.local/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [15]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(corpus_embeddings, labels, test_size=0.3, random_state=i)
    gnb = MLPClassifier(alpha=1, max_iter=1000)
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    print("Number of mislabeled points out of a total %d points : %d"
          % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 185 points : 13
Number of mislabeled points out of a total 185 points : 10
Number of mislabeled points out of a total 185 points : 12
Number of mislabeled points out of a total 185 points : 16
Number of mislabeled points out of a total 185 points : 14
Number of mislabeled points out of a total 185 points : 26
Number of mislabeled points out of a total 185 points : 8
Number of mislabeled points out of a total 185 points : 15
Number of mislabeled points out of a total 185 points : 13
Number of mislabeled points out of a total 185 points : 10
Number of mislabeled points out of a total 185 points : 16
Number of mislabeled points out of a total 185 points : 9
Number of mislabeled points out of a total 185 points : 16
Number of mislabeled points out of a total 185 points : 14
Number of mislabeled points out of a total 185 points : 11
Number of mislabeled points out of a total 185 points : 11
Number of mislabeled points out of a total 185 points : 13

In [19]:
X_test.shape

(185, 1024)

In [21]:
X_test[0,:].shape

(1024,)

In [27]:
gnb.predict_proba(X_test[2,:].reshape(1, -1))

array([[0.1488756, 0.8511244]], dtype=float32)

In [28]:
y_test[2]

0

In [ ]:
conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))
print('BAC: %.3f' % balanced_accuracy_score(y_test, y_pred))

In [29]:
corpus_embeddings.shape

(614, 1024)